In [1]:
library(ggplot2)
library(mgcv)
library(visreg)

Warning message:
“package ‘ggplot2’ was built under R version 3.6.2”
Warning message:
“package ‘mgcv’ was built under R version 3.6.2”
Loading required package: nlme

Warning message:
“package ‘nlme’ was built under R version 3.6.2”
This is mgcv 1.8-35. For overview type 'help("mgcv-package")'.

Warning message:
“package ‘visreg’ was built under R version 3.6.2”


In [2]:
datax=read.csv('gmdata.csv')

In [4]:
datay = datax[datax$CBF.TYPE =='CBF',]
pnc = datay[datay$Datasets=='PNC',]
nki = datay[datay$Datasets=='NKI',]
age = datay[datay$Datasets=='AGE',]
irr = datay[datay$Datasets=='IRR',]
ftd = datay[datay$Datasets=='FTD',]

#############################################################
cbf_Age_gam <- gam(CBF.GM ~ s(AGE, k=4), method="REML", data = datay)

#####################
## Look at results ##
#####################
summary(cbf_Age_gam)

## Nonlinear age effect
Age_pval <- summary(cbf_Age_gam)$s.table[1,4]
Age_pval

####################################
## Visualize Nonlinear Age Effect ##
####################################
plotdata <- visreg(cbf_Age_gam,'AGE',type = "conditional",scale = "linear", plot = FALSE)
smooths <- data.frame(Variable = plotdata$meta$x, 
                      x=plotdata$fit[[plotdata$meta$x]], 
                      smooth=plotdata$fit$visregFit, 
                      lower=plotdata$fit$visregLwr, 
                      upper=plotdata$fit$visregUpr)
predicts <- data.frame(Variable = "dim1", 
                       x=plotdata$res$AGE,
                       y=plotdata$res$visregRes)

CBF_Age_plot <- ggplot() + xlim(8, 85)+ ylim(0,140) +
  #theme(legend.position = "none")  +
  labs(x = "Age (years)", y = "CBF(mL/100 g/min)") +
  theme(axis.title.x = element_text(size = rel(1.6))) +
  theme(axis.title.y = element_text(size = rel(1.6))) + 
  theme(axis.text = element_text(size = rel(1.4))) + theme(axis.line = element_line(colour = 'black', size = 1.5), axis.ticks.length = unit(.25, "cm")) +
  theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(), panel.background = element_blank()) +
  geom_point(data = predicts, aes(x, y), colour = "darksalmon", alpha=0.7 ) +  
  #geom_point(data=datay,aes(x=AGE, y=CBF.GM),size=1)+ 
  geom_point(data=pnc,aes(x=AGE, y=CBF.GM),shape=21,size=2,fill='#e34a33', color="white")+ 
  geom_point(data=nki,aes(x=AGE, y=CBF.GM),shape=21,size=2,fill="#3182bd", color="white")+ 
  geom_point(data=irr,aes(x=AGE, y=CBF.GM),shape=21,size=2,fill="#31a354", color="white")+ 
  geom_point(data=ftd,aes(x=AGE, y=CBF.GM),shape=21,size=2,fill="#d95f02", color="white")+ 
  geom_point(data=age,aes(x=AGE, y=CBF.GM),shape=21,size=2,fill="#c51b8a", color="white")+ 
  geom_line(data = smooths, aes(x = x, y = smooth), colour = "midnightblue",size=1) +
  geom_line(data = smooths, aes(x = x, y=lower), linetype="dashed", colour = "midnightblue", alpha = 0.9, size = 0.9) + 
  geom_line(data = smooths, aes(x = x, y=upper), linetype="dashed",colour = "midnightblue", alpha = 0.9, size = 0.9) +
  coord_cartesian(xlim = c(10.5,85), ylim = c(0,140))
  
  
 

dev.off()

## Export image
png(filename="CBF_AGE_fit.png",res = 600, width = 4, height = 4, units = 'in',)
CBF_Age_plot
dev.off()



Family: gaussian 
Link function: identity 

Formula:
CBF.GM ~ s(AGE, k = 4)

Parametric coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  65.3339     0.2525   258.8   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Approximate significance of smooth terms:
        edf Ref.df    F p-value    
s(AGE) 2.99      3 1018  <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

R-sq.(adj) =  0.499   Deviance explained =   50%
-REML =  12420  Scale est. = 195.11    n = 3061

[1] 0

null device 
          1

Warning message:
“Removed 13 rows containing missing values (geom_point).”
Warning message:
“Removed 2 rows containing missing values (geom_point).”
Warning message:
“Removed 19 rows containing missing values (geom_point).”
Warning message:
“Removed 1 rows containing missing values (geom_point).”
Warning message:
“Removed 1 rows containing missing values (geom_point).”
Warning message:
“Removed 10 row(s) containing missing values (geom_path).”
Warning message:
“Removed 10 row(s) containing missing values (geom_path).”
Warning message:
“Removed 10 row(s) containing missing values (geom_path).”


null device 
          1

In [4]:
datay = datax[datax$CBF.TYPE =='BASIL',]
pnc = datay[datay$Datasets=='PNC',]
nki = datay[datay$Datasets=='NKI',]
age = datay[datay$Datasets=='AGE',]
irr = datay[datay$Datasets=='IRR',]
ftd = datay[datay$Datasets=='FTD',]

#############################################################
cbf_Age_gam <- gam(CBF.GM ~ s(AGE, k=4), method="REML", data = datay)

#####################
## Look at results ##
#####################
summary(cbf_Age_gam)

## Nonlinear age effect
Age_pval <- summary(cbf_Age_gam)$s.table[1,4]
Age_pval

####################################
## Visualize Nonlinear Age Effect ##
####################################
plotdata <- visreg(cbf_Age_gam,'AGE',type = "conditional",scale = "linear", plot = FALSE)
smooths <- data.frame(Variable = plotdata$meta$x, 
                      x=plotdata$fit[[plotdata$meta$x]], 
                      smooth=plotdata$fit$visregFit, 
                      lower=plotdata$fit$visregLwr, 
                      upper=plotdata$fit$visregUpr)
predicts <- data.frame(Variable = "dim1", 
                       x=plotdata$res$AGE,
                       y=plotdata$res$visregRes)

CBF_Age_plot <- ggplot() + xlim(10, 85)+ ylim(0,140) +
  #theme(legend.position = "none")  +
  labs(x = "Age (years)", y = "CBF(mL/100 g/min)") +
  theme(axis.title.x = element_text(size = rel(1.6))) +
  theme(axis.title.y = element_text(size = rel(1.6))) + 
  theme(axis.text = element_text(size = rel(1.4))) + theme(axis.line = element_line(colour = 'black', size = 1.5), axis.ticks.length = unit(.25, "cm")) +
  theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(), panel.background = element_blank()) +
  geom_point(data = predicts, aes(x, y), colour = "darksalmon", alpha=0.7 ) +  
  #geom_point(data=datay,aes(x=AGE, y=CBF.GM),size=1)+ 
  geom_point(data=pnc,aes(x=AGE, y=CBF.GM),shape=21,size=2,fill='#e34a33', color="white")+ 
  geom_point(data=nki,aes(x=AGE, y=CBF.GM),shape=21,size=2,fill="#3182bd", color="white")+ 
  geom_point(data=irr,aes(x=AGE, y=CBF.GM),shape=21,size=2,fill="#31a354", color="white")+ 
  geom_point(data=ftd,aes(x=AGE, y=CBF.GM),shape=21,size=2,fill="#d95f02", color="white")+ 
  geom_point(data=age,aes(x=AGE, y=CBF.GM),shape=21,size=2,fill="#c51b8a", color="white")+ 
  geom_line(data = smooths, aes(x = x, y = smooth), colour = "midnightblue",size=1) +
  geom_line(data = smooths, aes(x = x, y=lower), linetype="dashed", colour = "midnightblue", alpha = 0.9, size = 0.9) + 
  geom_line(data = smooths, aes(x = x, y=upper), linetype="dashed",colour = "midnightblue", alpha = 0.9, size = 0.9) +
  coord_cartesian(xlim = c(10.5,85), ylim = c(0,140))
  
  
 

dev.off()

## Export image
png(filename="BASIL_AGE_fit.png",res = 600, width = 4, height = 4, units = 'in',)
CBF_Age_plot
dev.off()


Family: gaussian 
Link function: identity 

Formula:
CBF.GM ~ s(AGE, k = 4)

Parametric coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  63.2795     0.2274   278.2   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Approximate significance of smooth terms:
         edf Ref.df     F p-value    
s(AGE) 2.995      3 800.5  <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

R-sq.(adj) =  0.439   Deviance explained =   44%
-REML =  12101  Scale est. = 158.32    n = 3061

[1] 0

null device 
          1

Warning message:
“Removed 229 rows containing missing values (geom_point).”
Warning message:
“Removed 149 rows containing missing values (geom_point).”
Warning message:
“Removed 88 rows containing missing values (geom_point).”
Warning message:
“Removed 1 rows containing missing values (geom_point).”
Warning message:
“Removed 1 rows containing missing values (geom_point).”
Warning message:
“Removed 12 row(s) containing missing values (geom_path).”
Warning message:
“Removed 12 row(s) containing missing values (geom_path).”
Warning message:
“Removed 12 row(s) containing missing values (geom_path).”


null device 
          1

In [5]:
datay = datax[datax$CBF.TYPE =='PVC',]
pnc = datay[datay$Datasets=='PNC',]
nki = datay[datay$Datasets=='NKI',]
age = datay[datay$Datasets=='AGE',]
irr = datay[datay$Datasets=='IRR',]
ftd = datay[datay$Datasets=='FTD',]

#############################################################
cbf_Age_gam <- gam(CBF.GM ~ s(AGE, k=4), method="REML", data = datay)

#####################
## Look at results ##
#####################
summary(cbf_Age_gam)

## Nonlinear age effect
Age_pval <- summary(cbf_Age_gam)$s.table[1,4]
Age_pval

####################################
## Visualize Nonlinear Age Effect ##
####################################
plotdata <- visreg(cbf_Age_gam,'AGE',type = "conditional",scale = "linear", plot = FALSE)
smooths <- data.frame(Variable = plotdata$meta$x, 
                      x=plotdata$fit[[plotdata$meta$x]], 
                      smooth=plotdata$fit$visregFit, 
                      lower=plotdata$fit$visregLwr, 
                      upper=plotdata$fit$visregUpr)
predicts <- data.frame(Variable = "dim1", 
                       x=plotdata$res$AGE,
                       y=plotdata$res$visregRes)

CBF_Age_plot <- ggplot() + xlim(8, 85)+ ylim(0,140) +
  #theme(legend.position = "none")  +
  labs(x = "Age (years)", y = "CBF(mL/100 g/min)") +
  theme(axis.title.x = element_text(size = rel(1.6))) +
  theme(axis.title.y = element_text(size = rel(1.6))) + 
  theme(axis.text = element_text(size = rel(1.4))) + theme(axis.line = element_line(colour = 'black', size = 1.5), axis.ticks.length = unit(.25, "cm")) +
  theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(), panel.background = element_blank()) +
  geom_point(data = predicts, aes(x, y), colour = "darksalmon", alpha=0.7 ) +  
  #geom_point(data=datay,aes(x=AGE, y=CBF.GM),size=1)+ 
  geom_point(data=pnc,aes(x=AGE, y=CBF.GM),shape=21,size=2,fill='#e34a33', color="white")+ 
  geom_point(data=nki,aes(x=AGE, y=CBF.GM),shape=21,size=2,fill="#3182bd", color="white")+ 
  geom_point(data=irr,aes(x=AGE, y=CBF.GM),shape=21,size=2,fill="#31a354", color="white")+ 
  geom_point(data=ftd,aes(x=AGE, y=CBF.GM),shape=21,size=2,fill="#d95f02", color="white")+ 
  geom_point(data=age,aes(x=AGE, y=CBF.GM),shape=21,size=2,fill="#c51b8a", color="white")+ 
  geom_line(data = smooths, aes(x = x, y = smooth), colour = "midnightblue",size=1) +
  geom_line(data = smooths, aes(x = x, y=lower), linetype="dashed", colour = "midnightblue", alpha = 0.9, size = 0.9) + 
  geom_line(data = smooths, aes(x = x, y=upper), linetype="dashed",colour = "midnightblue", alpha = 0.9, size = 0.9) +
 coord_cartesian(xlim = c(10.5,85), ylim = c(0,140))
  
  
 

dev.off()

## Export image
png(filename="PVC_AGE_fit.png",res = 600, width = 6, height = 4, units = 'in',)
CBF_Age_plot
dev.off()


Family: gaussian 
Link function: identity 

Formula:
CBF.GM ~ s(AGE, k = 4)

Parametric coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  69.1600     0.2688   257.3   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Approximate significance of smooth terms:
        edf Ref.df     F p-value    
s(AGE) 2.99      3 995.5  <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

R-sq.(adj) =  0.494   Deviance explained = 49.4%
-REML =  12611  Scale est. = 221.09    n = 3061

[1] 0

null device 
          1

Warning message:
“Removed 16 rows containing missing values (geom_point).”
Warning message:
“Removed 2 rows containing missing values (geom_point).”
Warning message:
“Removed 22 rows containing missing values (geom_point).”
Warning message:
“Removed 1 rows containing missing values (geom_point).”
Warning message:
“Removed 1 rows containing missing values (geom_point).”
Warning message:
“Removed 10 row(s) containing missing values (geom_path).”
Warning message:
“Removed 10 row(s) containing missing values (geom_path).”
Warning message:
“Removed 10 row(s) containing missing values (geom_path).”


null device 
          1

In [6]:
datay = datax[datax$CBF.TYPE =='SCRUB',]
pnc = datay[datay$Datasets=='PNC',]
nki = datay[datay$Datasets=='NKI',]
age = datay[datay$Datasets=='AGE',]
irr = datay[datay$Datasets=='IRR',]
#ftd = datay[datay$Datasets=='FTD',]

#############################################################
cbf_Age_gam <- gam(CBF.GM ~ s(AGE, k=4), method="REML", data = datay)

#####################
## Look at results ##
#####################
summary(cbf_Age_gam)

## Nonlinear age effect
Age_pval <- summary(cbf_Age_gam)$s.table[1,4]
Age_pval

####################################
## Visualize Nonlinear Age Effect ##
####################################
plotdata <- visreg(cbf_Age_gam,'AGE',type = "conditional",scale = "linear", plot = FALSE)
smooths <- data.frame(Variable = plotdata$meta$x, 
                      x=plotdata$fit[[plotdata$meta$x]], 
                      smooth=plotdata$fit$visregFit, 
                      lower=plotdata$fit$visregLwr, 
                      upper=plotdata$fit$visregUpr)
predicts <- data.frame(Variable = "dim1", 
                       x=plotdata$res$AGE,
                       y=plotdata$res$visregRes)

CBF_Age_plot <- ggplot() + xlim(8, 85)+ ylim(0,140) +
  #theme(legend.position = "none")  +
  labs(x = "Age (years)", y = "CBF(mL/100 g/min)") +
  theme(axis.title.x = element_text(size = rel(1.6))) +
  theme(axis.title.y = element_text(size = rel(1.6))) + 
  theme(axis.text = element_text(size = rel(1.4))) + theme(axis.line = element_line(colour = 'black', size = 1.5), axis.ticks.length = unit(.25, "cm")) +
  theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(), panel.background = element_blank()) +
  geom_point(data = predicts, aes(x, y), colour = "darksalmon", alpha=0.7 ) +  
  #geom_point(data=datay,aes(x=AGE, y=CBF.GM),size=1)+ 
  geom_point(data=pnc,aes(x=AGE, y=CBF.GM),shape=21,size=2,fill='#e34a33', color="white")+ 
  geom_point(data=nki,aes(x=AGE, y=CBF.GM),shape=21,size=2,fill="#3182bd", color="white")+ 
  geom_point(data=irr,aes(x=AGE, y=CBF.GM),shape=21,size=2,fill="#31a354", color="white")+ 
  #geom_point(data=ftd,aes(x=AGE, y=CBF.GM),shape=21,size=2,fill="#d95f02", color="white")+ 
  geom_point(data=age,aes(x=AGE, y=CBF.GM),shape=21,size=2,fill="#c51b8a", color="white")+ 
  geom_line(data = smooths, aes(x = x, y = smooth), colour = "midnightblue",size=1) +
  geom_line(data = smooths, aes(x = x, y=lower), linetype="dashed", colour = "midnightblue", alpha = 0.9, size = 0.9) + 
  geom_line(data = smooths, aes(x = x, y=upper), linetype="dashed",colour = "midnightblue", alpha = 0.9, size = 0.9) +
  coord_cartesian(xlim = c(10.5,85), ylim = c(0,140))
  
  
 

dev.off()

## Export image
png(filename="SCRUB_AGE_fit.png",res = 600, width = 6, height = 4, units = 'in',)
CBF_Age_plot
dev.off()


Family: gaussian 
Link function: identity 

Formula:
CBF.GM ~ s(AGE, k = 4)

Parametric coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  64.2066     0.2449   262.2   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Approximate significance of smooth terms:
         edf Ref.df     F p-value    
s(AGE) 2.992      3 996.7  <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

R-sq.(adj) =  0.503   Deviance explained = 50.4%
-REML =  11825  Scale est. = 176.86    n = 2950

[1] 0

null device 
          1

Warning message:
“Removed 10 rows containing missing values (geom_point).”
Warning message:
“Removed 19 rows containing missing values (geom_point).”
Warning message:
“Removed 1 rows containing missing values (geom_point).”
Warning message:
“Removed 3 row(s) containing missing values (geom_path).”
Warning message:
“Removed 3 row(s) containing missing values (geom_path).”
Warning message:
“Removed 3 row(s) containing missing values (geom_path).”


null device 
          1